In [1]:
%reload_ext autoreload
%autoreload 2

# Load the data

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [3]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm

DEVICE = "ibm_torino"
LOGICAL = str(0)
XBASIS = False
ROUNDS = 50

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
md = metadata_loader(True, True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]

# md = md[1:6]
print("shape:", md.shape)
md[:2]

state = Z0
shape: (40, 18)


,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,num_qubits,job_status,execution_date,code,distance,rounds,logical,descr,resets,xbasis,path_info
5483,2024-03-24 15:59:50.864000+01:00,Torino_RepCodes_jobs,ibm_torino,cr03zsqdvs8g008j7ra0,"[Subset 56, 50 rounds, 0 log, xbasis=False]",1.0,1398.0,NaN,JobStatus.DONE,2024-03-24 16:51:12.990431+01:00,RepetitionCodeCircuit,56,50,0,subset RepCodes,NaN,False,"{'mean_gate_error': 0.006452510204489, 'min_ga..."
5482,2024-03-24 15:59:44.592000+01:00,Torino_RepCodes_jobs,ibm_torino,cr03zr7k5z700081s2tg,"[Subset 56, 50 rounds, 0 log, xbasis=False]",1.0,1398.0,NaN,JobStatus.DONE,2024-03-24 16:50:44.865382+01:00,RepetitionCodeCircuit,56,50,0,subset RepCodes,NaN,False,"{'mean_gate_error': 0.006452510204489, 'min_ga..."


# Group job ids by closest calibration date

In [4]:
from datetime import datetime
from Scratch import find_closest_calib_jobs
import pandas as pd

jobs_by_calibration_date = {}
for index, row in md.iterrows():
    job_id = row['job_id']

    _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False)

    if calib_creation_date not in jobs_by_calibration_date.keys():
        jobs_by_calibration_date[calib_creation_date] = [job_id]
    else:
        jobs_by_calibration_date[calib_creation_date].append(job_id)

# Takes 15s

In [5]:
print(jobs_by_calibration_date)
print()
print(f"num of calibrations: {len(jobs_by_calibration_date)}")
print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")

{datetime.datetime(2024, 3, 23, 9, 58, 21, 614000, tzinfo=datetime.timezone.utc): ['cr03zsqdvs8g008j7ra0', 'cr03zr7k5z700081s2tg', 'cr03zppdvs8g008j7r90', 'cr03zn6dvs8g008j7r80', 'cr03zkp8gdp0008fxbtg', 'cr03zj6czq6g0081jwe0', 'cr03zgek5z700081s2r0', 'cr03zexdvs8g008j7r70', 'cr03zdds9z7g008dqshg', 'cr03zbxs9z7g008dqsh0', 'cr03zandvs8g008j7r6g', 'cr03z8xdvs8g008j7r60', 'cr03z7cdvs8g008j7r5g', 'cr03z5wdvs8g008j7r50', 'cr03z4ck5z700081s2pg', 'cr03z2wczq6g0081jwbg', 'cr03z1cdvs8g008j7r4g', 'cr03yzk8gdp0008fxbr0', 'cr03yybk5z700081s2ng', 'cr03ywk8gdp0008fxbqg'], datetime.datetime(2024, 3, 16, 15, 5, 33, 808000, tzinfo=datetime.timezone.utc): ['cqtvae1pkcdg008e3py0', 'cqtvachpkcdg008e3pxg', 'cqtvab988ev000813aeg', 'cqtva9s4x0mg008acne0', 'cqtva8988ev000813ae0', 'cqtva6r88ev000813ad0', 'cqtva10txzj0008y0a30', 'cqtv9zz9nfw0008ht1y0', 'cqtv9yq88ev000813ac0', 'cqtv9wq4x0mg008acncg', 'cqtv9vf9nfw0008ht1xg'], datetime.datetime(2024, 3, 16, 15, 3, 15, 271000, tzinfo=datetime.timezone.utc): ['cqtv9s

# Decode data

In [8]:
d = md["distance"].values[0]

distances = np.arange(7, d+1, 4)
distances = distances[::-1]
distances = [3]
distances

[3]

In [9]:
import json
import os
import time

import pymatching
import stim

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
import cpp_soft_info

SOFT_MUL = 0.5
file_name  = f'./results/CONV_{DEVICE}_{state}_{ROUNDS}_soft_{SOFT_MUL}.json'

rel_error = 1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
lin = [0.4, 0.7, 20]
lin = [0.4, 0.7, 1]
num_points = 51
bandwidths = np.linspace(lin[0], lin[1], lin[2])


for calib_date in jobs_by_calibration_date.keys():
    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date
    nb_shots_calib = None
    all_memories = load_calibration_memory(provider, tobecalib_backend=DEVICE, other_date=calib_date, nb_shots=nb_shots_calib)
    kde_dict = cpp_soft_info.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=51) # Less num_points bcs just 1 bandwidth

    # Retrieve the memories
    memories = []
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
        d = md[md["job_id"] == job_id]["distance"].values[0] 
        T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

        # Get the job
        job = provider.retrieve_job(job_id)
        memory = job.result().get_memory()
        memories.append(memory)

    # Stack the memories vertically
    big_memory = np.vstack(memories)

    # Get the layout of the last job (same as previous)
    layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
    link_qubits = list(layout_des['link_qubit'].values())
    code_qubits = list(layout_des['code_qubit'].values())

    # Get the pSoft and countMat matrices
    big_layout = link_qubits + code_qubits
    inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))
    print(f"Starting to get pSoft and countMat at {datetime.now()}")
    pSoft, countMat = cpp_soft_info.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1)
    print(f"Finished getting pSoft and countMat at {datetime.now()}")

    # Subsample decoding
    for D_NEW in tqdm(distances):
        subsets = generate_subsets_with_center(d, D_NEW)

        num_errors_kde = []
        num_errors_hard = []
        for subset in (subsets):             
            cols_to_keep = get_cols_to_keep(subset, T, d)
            pSoft_sub = pSoft[:, cols_to_keep]
            countMat_sub = countMat[:, cols_to_keep]

            # Get the layout
            layout = get_subsample_layout(subset, link_qubits, code_qubits)
            qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=T)

            # Get the noise avgs
            avgs = get_avgs_from_dict(noise_dict, layout)
            noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
            readout = avgs["readout"]
            noise_list += [readout, readout*(1-SOFT_MUL), readout*SOFT_MUL]    
            #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

            # Stim model
            subsampling = (D_NEW != d)
            code = RepetitionCodeStimCircuit(D_NEW, T, xbasis=XBASIS, resets=_RESETS, 
                                            noise_list=noise_list, subsampling=subsampling)
            model = code.circuits[LOGICAL].detector_error_model(decompose_errors=False)

            result_soft, result_hard = cpp_soft_info.decodeConvertorAll(model, countMat_sub, pSoft_sub, 
                                                                        T, int(LOGICAL), _RESETS)
            
            num_errors_kde.append(result_soft.num_errors)
            num_errors_hard.append(result_hard.num_errors)
    
        result_kde_json = {
            "decoding": "kde",
            "d_new": str(D_NEW),
            "num_errors": np.mean(num_errors_kde),
            "error_list": num_errors_kde,
            "additional_info": {
                "rel_error": rel_error,
                "bandwidth_linspace": lin,
                "num_points_bandwidths": num_points,
                "soft_multiplicator": SOFT_MUL,
            },
        }

        result_hard_json = {
            "decoding": "hard",
            "d_new": str(D_NEW),
            "num_errors": np.mean(num_errors_hard),
            "error_list": num_errors_hard,
            "noise_list": noise_list,
        }

        if not os.path.exists(file_name):
            data = {}
        else:
            with open(file_name, "r") as f:
                data = json.load(f)

        if job_id not in data.keys():
            data[job_id] = [result_kde_json, result_hard_json]
        else:
            data[job_id].append(result_kde_json)
            data[job_id].append(result_hard_json)
        
        with open(file_name, "w") as f:
            json.dump(data, f, indent=4)

21:51:18 Warning: Z0 decoding. Negative T2 error -0.06 % for qubit 90, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
21:51:21 Warning: Could not get two gate error of ECR due to 'Could not find the desired property for ecr', taking CX instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 67
21:51:21 Warning: Could not get two gate error of CX due to 'Could not find the desired property for cx', taking 0.5 instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 71
Found jobs for backend ibm_torino with closest execution date 2024-03-23 09:55:40.182587+00:00.


Retrieving jobs of 2024-03-23 09:58:21.614000+00:00 calibration: 100%|██████████| 20/20 [00:49<00:00,  2.47s/it]


Starting to get pSoft and countMat at 2024-03-25 21:52:21.347758
Finished getting pSoft and countMat at 2024-03-25 21:58:59.158307


100%|██████████| 1/1 [01:48<00:00, 108.86s/it]


22:00:50 Warning: Z0 decoding. Negative T2 error -0.00 % for qubit 12, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
22:00:53 Warning: Could not get two gate error of ECR due to 'Could not find the desired property for ecr', taking CX instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 67
22:00:53 Warning: Could not get two gate error of CX due to 'Could not find the desired property for cx', taking 0.5 instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 71
Found jobs for backend ibm_torino with closest execution date 2024-03-16 14:56:22.522112+00:00.


Retrieving jobs of 2024-03-16 15:05:33.808000+00:00 calibration: 100%|██████████| 11/11 [00:27<00:00,  2.49s/it]


Starting to get pSoft and countMat at 2024-03-25 22:01:31.241900
